In [1]:
import pandas as pd
from dotenv import load_dotenv, find_dotenv
import os
import pinecone
from pinecone import PodSpec

from canopy.tokenizer import Tokenizer

from tqdm.auto import tqdm
from canopy.knowledge_base import KnowledgeBase
from canopy.models.data_models import Document
from canopy.knowledge_base import list_canopy_indexes

from canopy.models.data_models import Query
from canopy.context_engine import ContextEngine
from canopy.models.data_models import UserMessage
import gradio as gr

In [2]:
import pandas as pd
from bs4 import BeautifulSoup

def html_to_text(text):
    soup = BeautifulSoup(text)
    return soup.get_text()
def clean_csv(filepath):
    df=pd.read_csv(filepath)
    df_filtered=df[~df["Body (HTML)"].isnull()][["Handle","Body (HTML)","SEO Description","Variant Price"]]
    df_filtered["Body Filtered HTML"]=df_filtered["Body (HTML)"].apply(html_to_text)
    df_filtered.to_parquet("products_export_1.parquet")

def append_source(text):
    return "https://contentia.cl/products/"+text

In [3]:
load_dotenv(find_dotenv())
PINECONE_API_KEY=os.getenv("PINECONE_API_KEY")
filepath_parquet=os.getenv("FILEPATH_PARQUET")
filepath_csv=os.getenv("FILEPATH_CSV")
gradio_port=os.getenv("PORT1")


In [5]:
data=pd.read_parquet(filepath_parquet)


In [6]:
data_for_upsert=data.rename(columns={"Handle":"id","Body Filtered HTML":"text"})[["id","text"]]
data_for_upsert

,id,text
0,bateria-np-f970-decodificada-para-luces-led-y-...,\nBatería NP-F970 de la marca Iluminus. \nCom...
1,cargador-de-baterias-serie-np-f-multifuncional...,Presentamos el cargador de baterías multifunci...
15,estabilizador-de-smartphone-zhiyun-smooth-q4-c...,\nEl estabilizador de cardán para smartphone ...
27,monopodo-de-video-xpro-de-4-secciones-con-base...,El último monopié XPRO de aluminio de 4 seccio...
30,monopodo-xpro-manfrotto-4-secciones-en-aluminio,El revolucionario monópodp XPRO de 4 secciones...
...,...,...
2224,vijim-vl196-luz-led-multicolor-con-difusor-y-p...,Gran rendimiento de color\nEl panel de luz LED...
2241,ulanzi-pt-12-adaptador-triple-zapata-fria,\nSoporte de zapata fría estándar universal\nC...
2248,ulanzi-st-16-soporte-de-telefonos-metalico-mul...,"Soporte metálico para teléfonos en trípode, co..."
2255,ulanzi-mt-11-tripode-flexible-para-camaras-reflex,El trípode Ulanzi MT-11 Octopus admite cámaras...


In [7]:
data_for_upsert["source"]=data_for_upsert["id"].apply(append_source)

In [8]:
data_for_upsert

,id,text,source
0,bateria-np-f970-decodificada-para-luces-led-y-...,\nBatería NP-F970 de la marca Iluminus. \nCom...,https://contentia.cl/products/bateria-np-f970-...
1,cargador-de-baterias-serie-np-f-multifuncional...,Presentamos el cargador de baterías multifunci...,https://contentia.cl/products/cargador-de-bate...
15,estabilizador-de-smartphone-zhiyun-smooth-q4-c...,\nEl estabilizador de cardán para smartphone ...,https://contentia.cl/products/estabilizador-de...
27,monopodo-de-video-xpro-de-4-secciones-con-base...,El último monopié XPRO de aluminio de 4 seccio...,https://contentia.cl/products/monopodo-de-vide...
30,monopodo-xpro-manfrotto-4-secciones-en-aluminio,El revolucionario monópodp XPRO de 4 secciones...,https://contentia.cl/products/monopodo-xpro-ma...
...,...,...,...
2224,vijim-vl196-luz-led-multicolor-con-difusor-y-p...,Gran rendimiento de color\nEl panel de luz LED...,https://contentia.cl/products/vijim-vl196-luz-...
2241,ulanzi-pt-12-adaptador-triple-zapata-fria,\nSoporte de zapata fría estándar universal\nC...,https://contentia.cl/products/ulanzi-pt-12-ada...
2248,ulanzi-st-16-soporte-de-telefonos-metalico-mul...,"Soporte metálico para teléfonos en trípode, co...",https://contentia.cl/products/ulanzi-st-16-sop...
2255,ulanzi-mt-11-tripode-flexible-para-camaras-reflex,El trípode Ulanzi MT-11 Octopus admite cámaras...,https://contentia.cl/products/ulanzi-mt-11-tri...


In [9]:
spec = PodSpec(environment="gcp-starter")

In [10]:
Tokenizer.initialize()
tokenizer = Tokenizer()
tokenizer.tokenize("Hello world!")


['Hello', ' world', '!']

In [12]:
index_name = "pinecone-docs"

kb = KnowledgeBase(index_name)

if not any(name.endswith(index_name) for name in list_canopy_indexes()):
    kb.create_canopy_index(spec=spec)

kb.connect()

In [16]:
documents = [Document(**row) for _, row in data_for_upsert.iterrows()]


In [17]:
batch_size = 100

for i in tqdm(range(0, len(documents), batch_size)):
    kb.upsert(documents[i: i+batch_size])

  0%|          | 0/4 [00:00<?, ?it/s]

In [13]:

context_engine = ContextEngine(kb)

from canopy.chat_engine import ChatEngine
chat_engine = ChatEngine(context_engine)


In [14]:
chat_history = [UserMessage(content="Hazme una lista de las grabadoras de audio disponible")]
print(chat_engine.chat(chat_history).choices[0].message.content)

Las grabadoras de audio disponibles son:

1. Grabadora de audio H1n con set de accesorios
2. Grabadora de audio portátil Zoom H4n Pro Black de 4 pistas
3. Grabadora de audio portátil Zoom H2n
4. Mixer de audio y grabadora multipista portátil LiveTrak L-8 Zoom
5. Grabadora y micrófono de mano de 4 canales Zoom M4
6. Grabadora de 8 canales Zoom H8
7. Grabadora de campo compacta Zoom F3

Estas son algunas de las grabadoras de audio actualmente disponibles en el mercado.


In [ ]:
another_entry = [UserMessage(content="tienes otros modelos para cargar baterias?")]
chat_engine.chat(another_entry).choices[0].message.content

In [ ]:
another_entry = [UserMessage(content="podrias hacer una lista de los bolsos disponibles?")]
print(chat_engine.chat(another_entry).choices[0].message.content)

In [4]:
def generate(input, output):
    return "R:"+input

demo = gr.Interface(fn=generate, 
                    inputs=[gr.Textbox(label="Prompt")], 
                    outputs=[gr.Textbox(label="Completion")])

gr.close_all()
demo.launch(share=True, server_port=int(os.environ['PORT1']))

/home/davidignacio/git/llm_examples/canopy_example/.venv/lib/python3.10/site-packages/gradio/utils.py:858: UserWarning: Expected 2 arguments for function <function generate at 0x7f3018dd9120>, received 1.
  warnings.warn(
/home/davidignacio/git/llm_examples/canopy_example/.venv/lib/python3.10/site-packages/gradio/utils.py:862: UserWarning: Expected at least 2 arguments for function <function generate at 0x7f3018dd9120>, received 1.
  warnings.warn(


Running on local URL:  http://127.0.0.1:8999
Running on public URL: https://a967784d15b03dbc0a.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


/home/davidignacio/git/llm_examples/canopy_example/.venv/lib/python3.10/site-packages/gradio/helpers.py:825: UserWarning: Unexpected argument. Filling with None.
  warnings.warn("Unexpected argument. Filling with None.")
